In [1]:

#-*-encoding:utf-8-*-
import requests
from bs4  import BeautifulSoup
import random
import json
import time
import thread

def cn_4Asms(r,url,parma,header,proxy):
        r.post(url,data=parma,headers=header,proxies=proxy) 
        thread.exit_thread()
        
def cn_4ALogin(r,isProxy=0,proxyIp='7.175.57.56',proxyPort='8083'):
    proxy={}
    if isProxy==1:
        proxy={'http':proxyIp+':'+proxyPort,'https':proxyIp+':'+proxyPort}
    
    header={'User-Agent':'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; WOW64; Trident/6.0; SE 2.X MetaSr 1.0)','Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8','Accept-Language':'zh-CN','Accept-Encoding':'gzip, deflate'}
    
    sendsms1='41608446240A6782F2A0F031426EDC066CF24674F3F0586A4D5FF056D75FF4AB'
#     sendsms1='D82167AC41A7716AD82167AC41A7716ACC7D552D8867DC74'
    #sendsms2会变的
    sendsms2='41608446240A6782F2A0F031426EDC066CF24674F3F0586AF1E3983438A09296B279671D1165AB116B295283AA6E45C8C790C1E0C84B5883'
#     sendsms2='D82167AC41A7716AD82167AC41A7716A8EEDA9B9710EE20D87FB9B882606933DBC8C75D72EB154316AD63E4B057A422F2285D2E95EB1600D'
    userInfo=sendsms2
    cn_name="shizhongxia"
    
    
    url='https://4a.gmcc.net'
    result1=r.get(url,headers=header,proxies=proxy,verify=False)
 
    #获取random_form
    soup1=BeautifulSoup(result1.text,'lxml')
    loginForm1=str(soup1.select('#loginForm')[0].select('input')[0]).split('=')[3].replace('"','').replace('/>','')
#     print loginForm1
    #登陆界面
    url='https://4acasp.gmcc.net/jk.do?method=checkUserType&userId='+cn_name
    a=r.post(url,data={'method':'checkUserType','userId':cn_name},headers=header,proxies=proxy,verify=False)

    #-----应该返回：SUCCESSG|TshizhongxiaG|T------
#     print a.text
    url='https://4acasp.gmcc.net/loginForward.do?target=https://4a.gmcc.net/first.do?method=login&appCode=IAM000&sendsms='+sendsms1
    parma={'random_form':loginForm1,'loginPage':'/auth/nextlogin.jsp','smsNameText':cn_name,'smsName':cn_name}
    result2=r.post(url,data=parma,headers=header,proxies=proxy,verify=False)
    soup2=BeautifulSoup(result2.text,'lxml')
    loginForm2=str(soup2.select('#loginForm')[0].select('input')[0]).split('=')[3].replace('"','').replace('/>','')
    parma={'sendsms':sendsms2}
    s=r.post('https://4acasp.gmcc.net/jk.do?method=checkSMSUser',data=parma,headers=header,proxies=proxy,verify=False)
    
    url='https://4acasp.gmcc.net/jk.do?method=checkSMSJK&target=https://4a.gmcc.net/first.do?method=login&authType=noteAuth&appCode=IAM000&loginPage=/auth/nextlogin.jsp&userInfo='+userInfo
    r.post(url,headers=header,proxies=proxy,verify=False)
    url='https://4acasp.gmcc.net/sendSms.do?smsfrom=send&target=https://4a.gmcc.net/first.do?method=login&authType=noteAuth&appCode=IAM000&loginPage=/auth/nextlogin.jsp'

    #下面这一步获取短信验证码
    thread.start_new_thread(cn_4Asms,(r,url,parma,header,proxy))
    
    print u'已经发送短信验证码'
    smsinput=raw_input(u'验证码:')
#     这里应该要做检查
    print u'正在登陆，请稍后...'

    # 验证
    url='https://4acasp.gmcc.net/loginServlet.do?target=https://4a.gmcc.net/first.do?method=login&authType=noteAuth&appCode=IAM000'
    parma={'random_form':loginForm2,'loginPage':'/auth/nextUserLogin.jsp','userName':cn_name,'passWord':'','tokenName':cn_name,'isUsePin':'true','updatePin':'false','tokenpassword':'','pinCode':'','smsName':cn_name,'smsPwd':'1234qweR','dynamic_smsPwd':smsinput,'checkCodeTemp':'','figerName':'','challengeName':cn_name,'challegePassword':'','vpn':''}
    result=r.post(url,data=parma,headers=header,proxies=proxy,verify=False)

    #信息同步
    random2='0.'+str(random.randint(100000000000000,999999999999999))+str(random.randint(1,9))
    s=r.get('https://4a.gmcc.net/synAuthz.do?method=syn&rnd='+random2,headers=header,proxies=proxy,verify=False)
    #-----{success:true,code:'null',text:'null'}---------

    #登陆4A
    random2='0.'+str(random.randint(100000000000000,999999999999999))+str(random.randint(1,9))
    r.get('https://4a.gmcc.net/main.jsp?rnd='+random2,headers=header,proxies=proxy,verify=False)
    r.get('https://4a.gmcc.net/SystemmsgPortlet.lp?id=systemmsg&userID='+cn_name+'&resType=1',headers=header,proxies=proxy)
    random2='0.'+str(random.randint(10000000000000000,99999999999999999))+str(random.randint(1,9))
    r.get('https://4a.gmcc.net/synAuthz.do?method=syn2AppServer&rnd='+random2,headers=header,proxies=proxy,verify=False)
    r.get('https://4a.gmcc.net/page/resgroup/searchFrame.jsp',headers=header,proxies=proxy,verify=False)
    r.post('https://4a.gmcc.net/keystore.do',data={'method':'getKeyStoreList'},headers=header,proxies=proxy,verify=False)
    #get:https://4a.gmcc.net/page/resource/appres/appresourceQuery.do?method=listIni
    r.post('https://4a.gmcc.net/page/resource/appres/appresourceQuery.do',data={'method':'pageQuery','_gt_json':'{"recordType":"object","pageInfo":{"pageSize":50,"pageNum":1,"totalRowNum":0,"totalPageNum":0,"startRowNum":0,"endRowNum":0},"columnInfo":[{"id":"id","header":"资源ID","fieldName":"id","fieldIndex":"id","sortOrder":null,"hidden":true,"exportable":true,"printable":true},{"id":"hasDueAcc","header":"hasDueAcc","fieldName":"hasDueAcc","fieldIndex":"hasDueAcc","sortOrder":null,"hidden":true,"exportable":true,"printable":true},{"id":"groupName","header":"资源组","fieldName":"groupName","fieldIndex":"groupName","sortOrder":null,"hidden":false,"exportable":true,"printable":true},{"id":"name","header":"资源名称","fieldName":"name","fieldIndex":"name","sortOrder":null,"hidden":false,"exportable":true,"printable":true},{"id":"ip","header":"IP地址","fieldName":"ip","fieldIndex":"ip","sortOrder":null,"hidden":false,"exportable":true,"printable":true},{"id":"restype","header":"资源类型","fieldName":"restype","fieldIndex":"restype","sortOrder":null,"hidden":true,"exportable":true,"printable":true},{"id":"appResAccountList","header":"从帐号列表","fieldName":"id","fieldIndex":"id","sortOrder":null,"hidden":false,"exportable":true,"printable":true},{"id":"ssoMode","header":"ssoMode","fieldName":"ssoMode","fieldIndex":"ssoMode","sortOrder":null,"hidden":"true","exportable":true,"printable":true},{"id":"sysResClientList","header":"登录","fieldName":"id","fieldIndex":"id","sortOrder":null,"hidden":false,"exportable":true,"printable":true},{"id":"opt","header":"操作","fieldName":"id","fieldIndex":"id","sortOrder":null,"hidden":false,"exportable":true,"printable":true}],"sortInfo":[],"filterInfo":[],"parameters":{},"action":"load"}'},headers=header,proxies=proxy,verify=False)
    r.post('https://4a.gmcc.net/keystore.do',data={'method':'getKeyStoreList'},headers=header,proxies=proxy,verify=False)
           
    params2={'method':'pageQuery','_gt_json':'{"recordType":"object","pageInfo":{"totalRowNum":-1,"pageSize":50,"pageNum":1,"totalPageNum":0,"startRowNum":0,"endRowNum":0},"columnInfo":[{"id":"id","header":"资源ID","fieldName":"id","fieldIndex":"id","sortOrder":null,"hidden":true,"exportable":true,"printable":true},{"id":"hasDueAcc","header":"hasDueAcc","fieldName":"hasDueAcc","fieldIndex":"hasDueAcc","sortOrder":null,"hidden":true,"exportable":true,"printable":true},{"id":"groupName","header":"资源组","fieldName":"groupName","fieldIndex":"groupName","sortOrder":null,"hidden":false,"exportable":true,"printable":true},{"id":"name","header":"资源名称","fieldName":"name","fieldIndex":"name","sortOrder":null,"hidden":false,"exportable":true,"printable":true},{"id":"ip","header":"IP地址","fieldName":"ip","fieldIndex":"ip","sortOrder":null,"hidden":false,"exportable":true,"printable":true},{"id":"restype","header":"资源类型","fieldName":"restype","fieldIndex":"restype","sortOrder":null,"hidden":true,"exportable":true,"printable":true},{"id":"appResAccountList","header":"从帐号列表","fieldName":"id","fieldIndex":"id","sortOrder":null,"hidden":false,"exportable":true,"printable":true},{"id":"ssoMode","header":"ssoMode","fieldName":"ssoMode","fieldIndex":"ssoMode","sortOrder":null,"hidden":"true","exportable":true,"printable":true},{"id":"sysResClientList","header":"登录","fieldName":"id","fieldIndex":"id","sortOrder":null,"hidden":false,"exportable":true,"printable":true},{"id":"opt","header":"操作","fieldName":"id","fieldIndex":"id","sortOrder":null,"hidden":false,"exportable":true,"printable":true}],"sortInfo":[],"filterInfo":[],"parameters":{},"action":"load"}','queryName':'','queryIP':'','isQuerySubTree':'1'}
    result2=r.post('https://4a.gmcc.net/page/resource/appres/appresourceQuery.do',data=params2,headers=header,proxies=proxy,verify=False)
    json2=json.loads(result2.text)
    return json2
           
def cn_json_load(json2,cn_system):
    cn_i=0
    for i in json2.get('data'):
        ss=i.get('name')
        if ss==cn_system:
            break
        cn_i=cn_i+1
    appResAccountList=json2.get('data')[cn_i].get('appResAccountList').split('|')[0][:-1]
    id_num=json2.get('data')[cn_i].get('id')
    return [appResAccountList,id_num]
           
            
def cn_BossLogin(r,json2,isProxy=0,proxyIp='7.175.57.56',proxyPort='8083'):
    if isProxy==0:
        proxy={}
    elif(isProxy==1):
        proxy={'http':proxyIp+':'+proxyPort,'https':proxyIp+':'+proxyPort}
        
    cn_subaccName='AGZGT0001511'
    header={'user-agent':'Mozilla/5.0 (Windows NT 6.2; WOW64; rv:42.0) Gecko/20100101 Firefox/42.0','Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8','Accept-Language':'en-US,en;q=0.5','Accept-Encoding':'gzip, deflate'}
    header2={'user-agent':'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; WOW64; Trident/6.0; SE 2.X MetaSr 1.0)','Accept':'image/jpeg,application/x-ms-application,image/gif,application/xaml+xml,image/pjpeg,application/x-ms-xbap,*/*','Accept-Language':'zh-CN','Accept-Encoding':'gzip, deflate'}
    #点击BOSS登陆
    cn_temp=cn_json_load(json2,u'广州NGBOSS前台')
    appResAccountList=cn_temp[0]
    id_num=cn_temp[1]
#     print appResAccountList
#     print id_num
    
    #同步数据
    r.post('https://4a.gmcc.net/page/resource/resourceQuery.do',data={'subaccName':cn_subaccName,'method':'isAuthDateValid'},headers=header,proxies=proxy,verify=False)
    params2={'method':'checkPolicy4JK','resId':id_num,'resAccId':appResAccountList,'date':str(time.time()).replace('.','')+str(random.randint(1,9))}
    r.post('https://4a.gmcc.net/page/jk/sso/appJKLogin.do',data=params2,headers=header,proxies=proxy,verify=False)
    params2={'method':'wantGetMACAddress','resId':id_num}
    r.post('https://4a.gmcc.net/sso.do',data=params2,headers=header,proxies=proxy,verify=False)
    
    s=r.get('https://4a.gmcc.net/page/portlet/resshow/macaddress.jsp?formUrl=/sso.do?method=appssoData&accID='+appResAccountList+'&resID='+id_num+'&softname=webAppByIE&date='+str(time.time()).replace('.','')+str(random.randint(1,9)),headers=header,proxies=proxy)
    
    url2='https://4a.gmcc.net/sso.do?method=appssoData&accID='+appResAccountList+'&resID='+id_num+'&softname=webAppByIE&date='+str(time.time()).replace('.','')+str(random.randint(1,9))
    params2={'txtMACAddr':'08:00:27:44:EC:99','txtIPAddr':'10.0.2.15','txtDNSName':'gzshizhongxia'}
    print '-----------测试是否登陆BOSS成功-----------'
    s1=r.post(url2,data=params2,headers=header2,proxies=proxy,allow_redirects=True,verify=False)
#     print s1.text
    

In [2]:

import requests


r=requests.session()
json2=cn_4ALogin(r)


/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/connectionpool.py:791: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/connectionpool.py:791: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/connectionpool.py:791: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/connectionpool.py:791: InsecureRequestWarning: Unverified H

已经发送短信验证码
验证码:159816


/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/connectionpool.py:791: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/connectionpool.py:791: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/connectionpool.py:791: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/connectionpool.py:791: InsecureRequestWarning: Unverified H

正在登陆，请稍后...


/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/connectionpool.py:791: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


In [3]:
cn_BossLogin(r,json2)


/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/connectionpool.py:791: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/connectionpool.py:791: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/connectionpool.py:791: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/connectionpool.py:791: InsecureRequestWarning: Unverified H

-----------测试是否登陆BOSS成功-----------


/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/connectionpool.py:791: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


In [ ]:
from bs4  import BeautifulSoup
import random
import time

def cn_cookies(cookies):
        cookies = dict()
        for i in r.cookies.get_dict('10.252.17.241'):
            cookies[i]=r.cookies.get(i,'','10.252.17.241')
        cookies['bsacKF']='NGCRM_BOSS'
        cookies['com.huawei.boss.CONTACTID']='null'
        cookies['com.huawei.boss.CURRENT_MENUID']='123456'
        cookies['com.huawei.boss.CURRENT_TAB']='123456'
        cookies['com.huawei.boss.CURRENT_USER']='com.huawei.boss.NO_CURRENT_USER'
        cookies['GZCOMM']='r_GZ_CRM_WEB1_1182'
        cookies['GZCRM']='r_GZ_CRM_WEB1_1081'
        return cookies

def cn_cookies1(cookies):
        cookies = dict()
        for i in r.cookies.get_dict('10.252.17.241'):
            cookies[i]=r.cookies.get(i,'','10.252.17.241')
        cookies['bsacKF']='NGCRM_BOSS'
        cookies['com.huawei.boss.CONTACTID']='undefined'
        cookies['com.huawei.boss.CURRENT_MENUID']='100110121062'
        cookies['com.huawei.boss.CURRENT_TAB']='BOSS%5E'+cn_phoneNum+'%5E100110121062%7E'+cn_phoneNum
        cookies['com.huawei.boss.CURRENT_USER']=cn_phoneNum
        cookies['GZCOMM']='r_GZ_CRM_WEB1_1182'
        cookies['GZCRM']='r_GZ_CRM_WEB1_1081'
        cookies['MACAddr']='08:00:27:A2:E7:39'
        cookies['sDNSName']='gzshizhongxia'
        return cookies

def cn_cookies2(cookies,cn_phoneNum):
        cookies = dict()
        for i in r.cookies.get_dict('10.252.17.241'):
            cookies[i]=r.cookies.get(i,'','10.252.17.241')
        cookies['bsacKF']='NGCRM_BOSS'
        cookies['com.huawei.boss.CONTACTID']='undefined'
        cookies['com.huawei.boss.CURRENT_MENUID']='CS_100010011003'
        cookies['com.huawei.boss.CURRENT_TAB']='BOSS%5E'+cn_phoneNum+'%5ECS_100010011003%7E'+cn_phoneNum
        cookies['com.huawei.boss.CURRENT_USER']=cn_phoneNum
        cookies['GZCOMM']='r_GZ_CRM_WEB1_1182'
        cookies['GZCRM']='r_GZ_CRM_WEB1_1181'
        cookies['GZPROTAL']='r_GZ_CRM_WEB2_1180'
        cookies['MACAddr']='08:00:27:A2:E7:39'
        cookies['sDNSName']='gzshizhongxia'
        return cookies
    
def cn_delete_entityid(r,cn_phoneNum,cn_pwd):
    cn_input_entityid=u'PV.SMS.0.8'
#     cn_phoneNum='15767570081'
#     cn_pwd='58b69c6e23b02e145a31494f2accd449f23ba7792e1f86e445d9c0f388636f153f75cc1d3a68d3e12294c0d56238348be59f3be3e77977df33355b1c4e2766e7767e4c9e3c5cdcb36ae32b809a1033dc0fe53b7b22f8f3441bea52e5c6123dd10643703f4e1a33656248d148f968ea5e5deece5eadb1983484844b0d102cf187'
    
    header={'User-Agent':'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; WOW64; Trident/6.0; SE 2.X MetaSr 1.0)','Accept':'*/*','Accept-Language':'zh-cn','Accept-Encoding':'gzip, deflate'}
    header2={'User-Agent':'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; WOW64; Trident/6.0; SE 2.X MetaSr 1.0)','Accept':'*/*','Accept-Language':'zh-cn','Accept-Encoding':'gzip, deflate','X-Requested-With':'XMLHttpRequest'}
    header3={'User-Agent':'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; WOW64; Trident/6.0; SE 2.X MetaSr 1.0)','Accept':'image/jpeg,application/x-ms-application,image/gif,application/xaml+xml,image/pjpeg, application/x-ms-xbap','Accept-Language':'zh-cn','Accept-Encoding':'gzip, deflate'}

    
    header={'User-Agent':'Mozilla/5.0 (Windows NT 6.2; WOW64; rv:42.0) Gecko/20100101 Firefox/42.0','Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8','Accept-Language':'en-US,en;q=0.5','Accept-Encoding':'gzip, deflate'}

    #获取主套餐
    parmas={}
    cookies=cn_cookies(r.cookies)
    result=r.post('http://10.252.17.241/ngcustcare/custlogin/getMainProID.action?servNumber='+cn_phoneNum,cookies=cookies,data=parmas,headers=header)
    cn_BASEPRODID=result.text
#     print cn_BASEPRODID
    # # ---------号码认证---------
    cookies=cn_cookies(r.cookies)
    parmas={'method':'qryCustInfo','servNumber':cn_phoneNum,'authCheckMode':'AuthCheckZ','verifyCode':'','pswd':cn_pwd,'certType':'IdCard','certID':'','rndPswd':'','custType':'PersonCustomer','domainType':'','isCert2G':'','ONLYLOGIN':'onlyLogin','withoutPassValidate':'true'}
    result=r.post('http://10.252.17.241/ngcustcare/custlogin/qryCustInfo.action',cookies=cookies,data=parmas,headers=header2)
    s=result.text
    cn_curCustId=s[s.find('curCustId')+12:s.find('curCustId')+25]
#     print cn_curCustId

    # # ---------获取最重要数据---------
#     print cn_phoneNum
    cookies=cn_cookies1(r.cookies)
    s=r.get('http://10.252.17.241/ngcustcare/uniteview/uviewtwo/uvDisper.action?currentTabID=BOSS^'+cn_phoneNum+'^100110121062~'+cn_phoneNum,cookies=cookies,headers=header3)
#     print s.text
    # # --------------------------
    
    

    #没显示到主界面
    cookies=cn_cookies2(r.cookies,cn_phoneNum)
    s=r.get('http://10.252.17.241/ngcustcare/product/changeProduct.action?currentTabID=BOSS^'+cn_phoneNum+'^CS_100010011003~'+cn_phoneNum,cookies=cookies,headers=header3)

    # 主界面
    cookies=cn_cookies2(r.cookies,cn_phoneNum)
    s=r.get('http://10.252.17.241/ngcustcare/product/productChangeAction!userProduct.action?BASEPRODID='+cn_BASEPRODID+'&RECTYPE=ChangeProduct&subsId='+cn_curCustId+'&KDSubRecType=',cookies=cookies,headers=header3)
    soup=BeautifulSoup(s.text,'lxml')
    #cn_CS_100010011003_productChangeAction获取
    temp=soup.select('input')
    temp1=str(temp[len(temp)-1])
    cn_CS_100010011003_productChangeAction=temp1[temp1.find('value')+7:temp1.find('value')+39]

#     print cn_CS_100010011003_productChangeAction
    #cn_objectID获取
    cn_objectid=''
    ss=soup.select('#userProductTable1')[0].select('.table_list')
    for i in ss:
        ss0=str(i)
        ss1=ss0.find('entityid')
        cn_entityid=ss0[ss1+10:].split('"')[0]
        if(cn_entityid==cn_input_entityid):
            ss2=ss0.find('objectid=')
            cn_objectid=ss0[ss2+10:].split('"')[0]
            break
    cn_objectID=cn_objectid
#     return cn_objectID
    if cn_objectID=='':
        return  '没有标识'
        
    return '有标识'
#     else:
#         print cn_objectID
#     ===这里应该要判断是否有可以删除的标识===#
#     print '123'
    #点击删除标识
    cn_random='0.'+str(random.randint(100000000000000,999999999999999))+str(random.randint(1,9))
    url='http://10.252.17.241/ngcustcare/product/productTreeAction!delUserProduct.action?storeRectype=ChangeProduct&maxCount=10&catalogName=&catalogId=&SPID=&SPBizID=&name=&goodsCatalogName=&goodsCatalogId=&appendCatalogSelect=ROOT_&searchId=&searchName=&searchShorname=&searchShor=&goodsCatalogSelect=ROOT_&goodsId=&goodsName=&goodsTypeSelect=&resId=&KDSubRecType=&OPPID=&SYSTEMTYPE=&RECTYPE=ChangeProduct&RECTYPE=&BASEPRODID='+cn_BASEPRODID+'&OLDBASEPRODID=&OBJECTID=&CHANNEL=bsacHal&CUSTTYPE=VC0000&SOLUTIONID=ALL&ROOTCATALOGID=ProdCatalogType_Person&EFFECTTYPE=&orderId=&hiddenTokenName=CS_100010011003_productChangeAction&CS_100010011003_productChangeAction='+cn_CS_100010011003_productChangeAction+'&objectID='+cn_objectID+'&random='+cn_random
    cookies=cn_cookies2(r.cookies)
    s=r.get(url,cookies=cookies,headers=header3)



#     #点击 关系检查
#     cn_random='0.'+str(random.randint(100000000000000,999999999999999))+str(random.randint(1,9))
#     url='http://10.252.17.241/ngcustcare/product/productChangeAction!prodRecCheck.action?storeRectype=ChangeProduct&maxCount=10&catalogName=&catalogId=&SPID=&SPBizID=&name=&goodsCatalogName=&goodsCatalogId=&appendCatalogSelect=ROOT_&searchId=&searchName=&searchShorname=&searchShor=&goodsCatalogSelect=ROOT_&goodsId=&goodsName=&goodsTypeSelect=&resId=&KDSubRecType=&OPPID=&SYSTEMTYPE=&RECTYPE=ChangeProduct&RECTYPE=&BASEPRODID='+cn_BASEPRODID+'&OLDBASEPRODID=&OBJECTID=&CHANNEL=bsacHal&CUSTTYPE=VC0000&SOLUTIONID=ALL&ROOTCATALOGID=ProdCatalogType_Person&EFFECTTYPE=&orderId=&hiddenTokenName=CS_100010011003_productChangeAction&CS_100010011003_productChangeAction='+cn_CS_100010011003_productChangeAction+'&random='+cn_random
#     cookies=cn_cookies2(r.cookies)
#     s=r.get(url,cookies=cookies,headers=header2)

#     cn_random='0.'+str(random.randint(100000000000000,999999999999999))+str(random.randint(1,9))
#     url='http://10.252.17.241/ngcustcare/product/productChangeAction!showProdEffectTime.action?type=change&random='+cn_random
#     cookies=cn_cookies2(r.cookies)
#     s=r.get(url,cookies=cookies,headers=header2)


#     time.sleep(2)
#     #点击确认操作
#     url='http://10.252.17.241/ngcustcare/product/productCommitAction!commit.action?RECTYPE=%27ChangeProduct%27&busiType=&orderId=&hiddenTokenName=CS_100010011003_productChangeAction&CS_100010011003_productChangeAction='+cn_CS_100010011003_productChangeAction
#     cookies=cn_cookies2(r.cookies)
#     s=r.get(url,cookies=cookies,headers=header3)
#     time.sleep(2)
#     #获取cn_CS_100010011003_fee
#     soup=BeautifulSoup(s.text,'lxml')
    
#     cn_CS_100010011003_fee=soup.find(attrs={"name":"CS_100010011003_fee/calculate"}).attrs['value']
    


#     #提交操作
#     url='http://10.252.17.241/ngcustcare/common/fee/pay.action'
#     cookies=cn_cookies2(r.cookies)
#     parmas={'PreBussOrderId':'','withdrawGoods':'','payTypeBean.cashAmt':'','payTypeBean.chequeAmt':'','payTypeBean.bankId':'','payTypeBean.bankName':'','payTypeBean.chequeNum':'','payTypeBean.cardAmt':'','payTypeBean.posNum':'','payTypeBean.confirmPosNum':'','payTypeBean.acctCash':'','payTypeBean.acctId':'-1','payTypeBean.scoreCash':'','payTypeBean.score':'','payTypeBean.scoreTransRule':'','payTypeBean.m':'0','payTypeBean.mTransRule':'','payTypeBean.bankAmt':'','payTypeBean.limitCash':'','payTypeBean.unlimitCash':'','BARGAINFEEXML':'','CHANGEFEEBYBARGAINXML':'@260@263xml@232version@261@2341@2460@234@232encoding@261@234UTF@2458@234@263@262@260huawei@295call@262@260i@262common@247fee@247@242@260@247i@262@260e@262changeFeeByBargain@260@247e@262@260p@262@260m@262@260n@2620@260@247n@262@260t@262a@260@247t@262@260v@262@260@247v@262@260@247m@262@260m@262@260n@2621@260@247n@262@260t@262a@260@247t@262@260v@262@260@247v@262@260@247m@262@260@247p@262@260@247huawei@295call@262','UPDATEPAYTYPEBYPAGEDATAXML':'@260@263xml@232version@261@2341@2460@234@232encoding@261@234UTF@2458@234@263@262@260huawei@295call@262@260i@262common@247fee@247@242@260@247i@262@260e@262updatePayTypeByPageData@260@247e@262@260p@262@260m@262@260n@2620@260@247n@262@260t@262a@260@247t@262@260v@262@260@247v@262@260@247m@262@260@247p@262@260@247huawei@295call@262','reccustinfo_name':u'施仲夏','reccustinfo_phone':'13922204911','reccustinfo_certificateType':'IdCard','reccustinfo_certificateNum':'440103198607251838','reccustinfo_address':u'广州越秀区天河路13号','reccustinfo_note':'','consigner_linkman':'','consigner_linkphone':'','consigner_id':'','consigner_address':'','consigner_postcode':'','invoicePrintMode':'','hiddenTokenName':'CS_100010011003_fee/calculate','CS_100010011003_fee/calculate':cn_CS_100010011003_fee,'receiptNumber':'00000000','invoiceType':'1','assembleInvoice':'0','invoiceNumber':'00000000','isForPaging':'NEW','changeEnumRecType':'',}
#     s=r.post(url,data=parmas,cookies=cookies,headers=header3)
#     soup=BeautifulSoup(s.text,'lxml')
#     return ''.join(soup.select('#error_msg_title')[0].text.split())

In [17]:
#看出号码状态,品牌
from bs4  import BeautifulSoup
import random

def cn_phone_band(cn_phoneNum):
    header={'User-Agent':'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; WOW64; Trident/6.0; SE 2.X MetaSr 1.0)','Accept':'*/*','Accept-Language':'zh-cn','Accept-Encoding':'gzip, deflate'}
    cn_random='0.'+str(random.randint(1000000000000000,9999999999999999))+str(random.randint(1,9))
    result=r.post('http://10.252.17.241/ngcustcare/chargesrv/common/relateSubs.action?servNumber='+cn_phoneNum+'&isSupportGrp=undefined&random='+cn_random,headers=header)
    soup=BeautifulSoup(result.text,'lxml')
    abc=soup.find(attrs={"name":"brand"}).attrs['value']+' '+soup.select('.mw_message')[1].text
    return abc

status=cn_phone_status1('13922204911')
print status

BrandGotone 施*夏


In [13]:
#看出号码状态
from bs4  import BeautifulSoup
import random

def cn_phone_status(cn_phoneNum):
    header={'User-Agent':'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; WOW64; Trident/6.0; SE 2.X MetaSr 1.0)','Accept':'*/*','Accept-Language':'zh-cn','Accept-Encoding':'gzip, deflate'}
    cn_random='0.'+str(random.randint(1000000000000000,9999999999999999))+str(random.randint(1,9))
    result=r.post('http://10.252.17.241/ngcustcare/chargesrv/common/relateSubs.action?servNumber='+cn_phoneNum+'&isSupportGrp=undefined&random='+cn_random,headers=header)
    soup=BeautifulSoup(result.text,'lxml')
    return soup.find(attrs={"name":"status"}).attrs['value']



US10


In [19]:
import time
from datetime import datetime

cn_phonNnum=''
# cn_list='15767479601;15767480541'

cn_list='13889905512;18818852008;13889902151;13889908829;13926079000;18818854588;18818858602;13889904059;18818858508;15802025610;18818852097;13570241618;13694260509;13711266833;13802993981;13889905988;13751767010;15800237189;13632210526;13678922399;15989007425;13929516532;13662483522;18818852027;15915778180;13632450336;13640806557;15915814784;13710072207;13699745105;13650881702;18818858581;18818858578;13450264242;13809241699;18819339417;18818854450;18818856919;13751864119;13570374684;13580340072;18818855918;15817125309;13889905517;13711707978;13711707058'
a=cn_list.split(';')
cn_a_count=len(a)
print cn_a_count
i=0
for cn_phonNnum in a:
    time.sleep(1)
    startTime=datetime.now()
    i=i+1
    try:
        status=cn_phone_status1(cn_phonNnum)
        print i,cn_phonNnum,status
    except:
        endTime=datetime.now()-startTime
        print i,cn_phonNnum,'status环节错误',endTime
        continue

46
1 13889905512 BrandGotone 江*舟
2 18818852008 BrandGotone 徐*领
3 13889902151 BrandGotone *鑫
4 13889908829 BrandGotone 王*筱
5 13926079000 BrandGotone 杨*村
6 18818854588 BrandGotone 何*伟
7 18818858602 BrandGotone *静
8 13889904059 BrandGotone 甄*立
9 18818858508 BrandGotone 马*琳
10 15802025610 BrandMzone *艳
11 18818852097 BrandGotone 罗*灿
12 13570241618 BrandMzone 尹*园
13 13694260509 BrandSzx 黄*新
14 13711266833 BrandGotone *玲
15 13802993981 BrandGotone 胡*婷
16 13889905988 BrandGotone *萍
17 13751767010 BrandMzone 刘*峰
18 15800237189 BrandSzx 高*宪
19 13632210526 BrandSzx *健
20 13678922399 BrandSzx *涛
21 15989007425 BrandMzone *晗
22 13929516532 BrandGotone *瞻
23 13662483522 BrandMzone 何*君
24 18818852027 BrandGotone 朱*峰
25 15915778180 BrandMzone 新*户
26 13632450336 BrandMzone 李*云
27 13640806557 BrandSzx *敏
28 15915814784 BrandMzone *茂
29 13710072207 BrandGotone 范*花
30 13699745105 BrandSzx 杨*璐
31 13650881702 BrandGotone 张*武
32 18818858581 BrandGotone 乔*清
33 18818858578 BrandGotone *凯
34 13450264242 BrandS

In [ ]:
# 删除标识
import time
cn_pwd='58b69c6e23b02e145a31494f2accd449f23ba7792e1f86e445d9c0f388636f153f75cc1d3a68d3e12294c0d56238348be59f3be3e77977df33355b1c4e2766e7767e4c9e3c5cdcb36ae32b809a1033dc0fe53b7b22f8f3441bea52e5c6123dd10643703f4e1a33656248d148f968ea5e5deece5eadb1983484844b0d102cf187'

b=u'15768906552;15768906553;15768906554;15768906556;15768906557;15768906558;15768906560;15768906561;15768906562'

#判断如果只有一个号码而没有';'，要如何操作

b0=b.split(';')
sum_num=str(len(b0))
temp_i=1
for cn_phoneNum in b0:
    print str(temp_i)+'/'+sum_num,cn_phoneNum,cn_delete_entityid(r,cn_phoneNum,cn_pwd),cn_liuliang(r,cn_phoneNum,cn_pwd)
    
    time.sleep(3)
    temp_i=temp_i+1
print 'end'

In [ ]:
from bs4  import BeautifulSoup
import random
import time


def cn_ChaYuE(r,cn_phonNum):
    header={'User-Agent':'Mozilla/5.0 (Windows NT 6.2; WOW64; rv:42.0) Gecko/20100101 Firefox/42.0','Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8','Accept-Language':'en-US,en;q=0.5','Accept-Encoding':'gzip, deflate'}

    s=r.get('http://10.252.17.241/ngcustcare/chargesrv/common/relateSubs.action?servNumber='+cn_phonNum+'&isSupportGrp=undefined&random=0.'+str(random.randint(100000000000000,999999999999999))+str(random.randint(1,9)),headers=header)
    soup=BeautifulSoup(s.text,'lxml')
    for i in soup.select('.table_list02')[0].select('tr'):

            temp=i.select('td')
    #         print ''.join(temp[3].text.split())
            if (''.join(temp[3].text.split())!=u'未生效资料') and (''.join(temp[3].text.split())!=u'用户状态'):
                try:
                    cn_subid_temp1=str(temp[1].select('input')[1])
                    cn_subid_begin=cn_subid_temp1.find('value="')
                    cn_subsId=cn_subid_temp1[cn_subid_begin+7:cn_subid_begin+20]
                except:
                    pass   
    #             查询余额
                params={'searchNumber':cn_phonNum,'subsId':cn_subsId,'balanceType':'CanUse','searchType':'servNumber'}
                url='http://10.252.17.241/ngcustcare/chargesrv/accountInfoQry/balanceQry/query.action'
                s=r.post(url,data=params,headers=header)
                soup=BeautifulSoup(s.text,'lxml')
                temp1=soup.select('#columnDivBalanceQry')[0].select('.table_list03')[0].select('tr')
                print ''.join(temp[1].text.split()),''.join(temp[2].text.split()),''.join(temp[3].text.split()),''.join(temp[4].text.split()),cn_subsId,''.join(temp1[3].select('td')[2].text.split()),''.join(temp1[3].select('td')[3].text.split()),''.join(temp1[4].select('td')[0].text.split()),''.join(temp1[4].select('td')[1].text.split())
                break

cn_phonNum=u'15767570081'

for i in cn_phonNum.split(';'):
    cn_ChaYuE(r,i)
    time.sleep(3)

In [ ]:
# 查流量剩余

from bs4  import BeautifulSoup
import random
import time


def cn1_cookies(cookies):
        cookies = dict()
        for i in r.cookies.get_dict('10.252.17.241'):
            cookies[i]=r.cookies.get(i,'','10.252.17.241')
        cookies['bsacKF']='NGCRM_BOSS'
        cookies['com.huawei.boss.CONTACTID']='null'
        cookies['com.huawei.boss.CURRENT_MENUID']='123456'
        cookies['com.huawei.boss.CURRENT_TAB']='123456'
        cookies['com.huawei.boss.CURRENT_USER']='com.huawei.boss.NO_CURRENT_USER'
        cookies['GZCOMM']='r_GZ_CRM_WEB1_1182'
        cookies['GZCRM']='r_GZ_CRM_WEB1_1081'
        return cookies

def cn1_cookies1(cookies,cn_phoneNum):
        cookies = dict()
        for i in r.cookies.get_dict('10.252.17.241'):
            cookies[i]=r.cookies.get(i,'','10.252.17.241')
        cookies['bsacKF']='NGCRM_BOSS'
        cookies['com.huawei.boss.CONTACTID']='undefined'
        cookies['com.huawei.boss.CURRENT_MENUID']='100110121062'
        cookies['com.huawei.boss.CURRENT_TAB']='BOSS%5E'+cn_phoneNum+'%5E100110121062%7E'+cn_phoneNum
        cookies['com.huawei.boss.CURRENT_USER']=cn_phoneNum
        cookies['GZCOMM']='r_GZ_CRM_WEB1_1082'
        cookies['GZCRM']='r_GZ_CRM_WEB1_1081'
        cookies['MACAddr']='08:00:27:A2:E7:39'
        cookies['sDNSName']='gzshizhongxia'
        return cookies
    
    
def cn1_cookies2(cookies,cn_phoneNum):
        cookies = dict()
        for i in r.cookies.get_dict('10.252.17.241'):
            cookies[i]=r.cookies.get(i,'','10.252.17.241')
        cookies['bsacKF']='NGCRM_BOSS'
        cookies['com.huawei.boss.CONTACTID']='undefined'
        cookies['com.huawei.boss.CURRENT_MENUID']='100110121062'
        cookies['com.huawei.boss.CURRENT_TAB']='BOSS%5E'+cn_phoneNum+'%5E100110121062%7E'+cn_phoneNum
        cookies['com.huawei.boss.CURRENT_USER']=cn_phoneNum
        cookies['GZCOMM']='r_GZ_CRM_WEB1_1082'
        cookies['GZCRM']='r_GZ_CRM_WEB2_1181'
        cookies['MACAddr']='08:00:27:A2:E7:39'
        cookies['sDNSName']='gzshizhongxia'
        cookies['GZPORTAL']='r_GZ_CRM_WEB2_1180'
        cookies['LIFE_TIME']='1461917029476_120'
        cookies['loginTicketCookie']='0557143B8C540DFD4EBE82042E6358C4D3213E73EDA9C34A706E178FC53865BD'
        return cookies
    
# 查流量剩余
def cn_liuliang(r,cn_phoneNum,cn_pwd):
    
    header={'User-Agent':'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; WOW64; Trident/6.0; SE 2.X MetaSr 1.0)','Accept':'*/*','Accept-Language':'zh-cn','Accept-Encoding':'gzip, deflate'}
    header2={'User-Agent':'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; WOW64; Trident/6.0; SE 2.X MetaSr 1.0)','Accept':'*/*','Accept-Language':'zh-cn','Accept-Encoding':'gzip, deflate','X-Requested-With':'XMLHttpRequest'}
    header3={'User-Agent':'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; WOW64; Trident/6.0; SE 2.X MetaSr 1.0)','Accept':'image/jpeg,application/x-ms-application,image/gif,application/xaml+xml,image/pjpeg, application/x-ms-xbap','Accept-Language':'zh-cn','Accept-Encoding':'gzip, deflate'}

    
    header={'User-Agent':'Mozilla/5.0 (Windows NT 6.2; WOW64; rv:42.0) Gecko/20100101 Firefox/42.0','Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8','Accept-Language':'en-US,en;q=0.5','Accept-Encoding':'gzip, deflate'}

    #获取主套餐
    parmas={}
    cookies=cn1_cookies(r.cookies)
    result=r.post('http://10.252.17.241/ngcustcare/custlogin/getMainProID.action?servNumber='+cn_phoneNum,cookies=cookies,data=parmas,headers=header)
    cn_BASEPRODID=result.text
#     print cn_BASEPRODID
    # # ---------号码认证---------
    cookies=cn1_cookies(r.cookies)
    parmas={'method':'qryCustInfo','servNumber':cn_phoneNum,'authCheckMode':'AuthCheckZ','verifyCode':'','pswd':cn_pwd,'certType':'IdCard','certID':'','rndPswd':'','custType':'PersonCustomer','domainType':'','isCert2G':'','ONLYLOGIN':'onlyLogin','withoutPassValidate':'true'}
    result=r.post('http://10.252.17.241/ngcustcare/custlogin/qryCustInfo.action',cookies=cookies,data=parmas,headers=header2)
    s=result.text
    cn_curCustId=s[s.find('curCustId')+12:s.find('curCustId')+25]
#     print cn_curCustId

    # # ---------获取最重要数据---------
#     print cn_phoneNum
    cookies=cn1_cookies1(r.cookies,cn_phoneNum)
    s=r.get('http://10.252.17.241/ngcustcare/uniteview/uviewtwo/uvDisper.action?currentTabID=BOSS^'+cn_phoneNum+'^100110121062~'+cn_phoneNum,cookies=cookies,headers=header3)
#     print s.text
    # # --------------------------
    soup=BeautifulSoup(s.text,'lxml')
#     print soup
    cn_ccm_EntityID=soup.find(attrs={"name":"ccm_EntityID"}).attrs['value']
    cn_ccm_EntityName=soup.find(attrs={"name":"ccm_EntityName"}).attrs['value']
    cn_ccm_CreateDate=soup.find(attrs={"name":"ccm_CreateDate"}).attrs['value']
    cn_ccm_Status=soup.find(attrs={"name":"ccm_Status"}).attrs['value']
    cn_ccm_StatusDate=soup.find(attrs={"name":"ccm_StatusDate"}).attrs['value']
    cn_ccm_EditStatus=soup.find(attrs={"name":"ccm_EditStatus"}).attrs['value']
    cn_ccm_Password=soup.find(attrs={"name":"ccm_Password"}).attrs['value']
    cn_csm_EntityID=soup.find(attrs={"name":"csm_EntityID"}).attrs['value']
    cn_csm_EntityName=soup.find(attrs={"name":"csm_EntityName"}).attrs['value']
    cn_csm_Status=soup.find(attrs={"name":"csm_Status"}).attrs['value']
    cn_csm_ProductID=soup.find(attrs={"name":"csm_ProductID"}).attrs['value']
    cn_csm_AccountOid=soup.find(attrs={"name":"csm_AccountOid"}).attrs['value']
    cn_csm_CustomerOid=soup.find(attrs={"name":"csm_CustomerOid"}).attrs['value']
    cn_csm_StopKey=soup.find(attrs={"name":"csm_StopKey"}).attrs['value']
    cn_csm_StartDate=soup.find(attrs={"name":"csm_StartDate"}).attrs['value']
    cn_csm_InvalidDate=soup.find(attrs={"name":"csm_InvalidDate"}).attrs['value']
    cn_csm_Enum=soup.find(attrs={"name":"csm_Enum"}).attrs['value']
    cn_csm_Imsi=soup.find(attrs={"name":"csm_Imsi"}).attrs['value']
    cn_cam_EntityID=soup.find(attrs={"name":"cam_EntityID"}).attrs['value']
    cn_cam_EntityName=soup.find(attrs={"name":"cam_EntityName"}).attrs['value']
    cn_cam_CreateDate=soup.find(attrs={"name":"cam_CreateDate"}).attrs['value']
    cn_cam_Status=soup.find(attrs={"name":"cam_Status"}).attrs['value']
    cn_cam_StatusDate=soup.find(attrs={"name":"cam_StatusDate"}).attrs['value']
    cn_cam_CustID=soup.find(attrs={"name":"cam_CustID"}).attrs['value']
    cn_cam_CreateDateDisplay=soup.find(attrs={"name":"cam_CreateDateDisplay"}).attrs['value']
    cn_com_CreateDate=soup.find(attrs={"name":"com_CreateDate"}).attrs['value']
    cn_com_StatusDate=soup.find(attrs={"name":"com_StatusDate"}).attrs['value']
    cn_com_Password=soup.find(attrs={"name":"com_Password"}).attrs['value']
    s2=r.get('http://10.252.17.241/nguniteview/bossviewhome.jsp?ccm_ObjectID=&ccm_RandomNum=&ccdirect=false&ccm_EntityID='+cn_ccm_EntityID+'&ccm_EntityName='+cn_ccm_EntityName+'&ccm_CreateDate='+cn_ccm_CreateDate+'&ccm_Status='+cn_ccm_Status+'&ccm_StatusDate='+cn_ccm_StatusDate+'&ccm_EditStatus='+cn_ccm_EditStatus+'&ccm_Region=200&ccm_ShortName=&ccm_Password='+cn_ccm_Password+'&ccm_CustType=PersonCustomer&ccm_VipType=VC0000&ccm_Foreigner=0&ccm_CustClass1=CustClass101&ccm_CustClass2=CustClass201&ccm_National=&ccm_Address=&ccm_CertID=&ccm_CertType=UnitID&ccm_CertAddr=&ccm_LinkMan=&ccm_LinkPhone=&ccm_HomeTel=&ccm_OfficeTel=&ccm_MobileTel=&ccm_PostCode=&ccm_LinkAddr=&ccm_Email=&ccm_HomePage=&ccm_IsMergeBill=0&ccm_CreditLevel=crdtNormal&ccm_OwnerAreaID=&ccm_OrgID=AAGZ0TZZI&ccm_RegStatus=1&ccm_Notes=&ccm_ResponseCustMgr=&ccm_CurrentCustMgr=&ccm_InLevel=1&ccm_TownID=&ccm_VipTypeStateDate=&ccm_NetServGrade=&ccm_CustAddrArray=%5B%5D&ccm_CustBillArray=%5B%5D&ccm_IsEncrypt=1&ccm_starLevel=&ccregister=true&ccnotRegister=false&csm_ObjectID=&csm_RandomNum=&csdirect=false&csm_EntityID='+cn_csm_EntityID+'&csm_EntityName='+cn_csm_EntityName+'&csm_CreateDate=******&csm_Status='+cn_csm_Status+'&csm_StatusDate=******&csm_EditStatus=0&csm_Region=200&csm_Password=&csm_ProductID='+cn_csm_ProductID+'&csm_ServNumber='+cn_phoneNum+'&csm_RegisterOrgID=AAGZ0TZZI&csm_OwnerOrgID=AAGZ0TZZI&csm_AccountOid='+cn_csm_AccountOid+'&csm_CustomerOid='+cn_csm_CustomerOid+'&csm_AuthType=AuthCheckB&csm_MobileType=mbtpNml&csm_AreaID=&csm_PayMode=2010&csm_StopKey='+cn_csm_StopKey+'&csm_Brand=BrandGotone&csm_StartDate='+cn_csm_StartDate+'&csm_InvalidDate='+cn_csm_InvalidDate+'&csm_SettleDay=1&csm_Enum='+cn_csm_Enum+'&csm_Imsi='+cn_csm_Imsi+'&csm_Credit=0&csm_SubsProduct=%5B%5D&csm_IsDefaultTelNum=1&csm_BelongRegion=0&csm_TargetRegion=0&csm_CretLevel=&csgotone=true&csnotGotone=false&cam_ObjectID=&cam_RandomNum=&cadirect=false&cam_EntityID='+cn_cam_EntityID+'&cam_EntityName='+cn_cam_EntityName+'&cam_CreateDate='+cn_cam_CreateDate+'&cam_Status='+cn_cam_Status+'&cam_StatusDate='+cn_cam_StatusDate+'&cam_EditStatus=0&cam_Region=200&cam_CustID='+cn_cam_CustID+'&cam_GroupAcctID=0&cam_PrePayType=pptpPost&cam_AccountType=actpCmn&cam_OverDraft=0&cam_ControlScheme=0&cam_EntrustTel=&cam_OrgID=AAGZ0TZZI&cam_NotifyType=1&cam_NotifyValue=0&cam_InvPrintType=&cam_Notes=&cam_PayChannelArray=%5B%5D&cam_CustBillArray=%5B%5D&cam_UrgeInfos=%5B%5D&cam_SettleAccount=&cam_IsDefault=1&cam_BillInvoiceMailArray=%5B%5D&cam_NotifyValueDisplay=0.00&cam_CreateDateDisplay='+cn_cam_CreateDateDisplay+'&com_ObjectID=&com_RandomNum=&comenuId=123456&codirect=false&com_EntityID=AGZGT0000829&com_EntityName=%CA%A9%D6%D9%CF%C4&com_CreateDate='+cn_com_CreateDate+'&com_Status=1&com_StatusDate='+cn_com_StatusDate+'&com_EditStatus=0&com_Region=200&conodeID=AGZGT0000829&conodeName=%CA%A9%D6%D9%CF%C4&conodeParentID=AAGZ0TZZ4&coentityID=AGZGT0000829&cootherParam=OPRT10&cochecked=false&com_MenuID=123456&com_ServNum=&com_TouchNum=&com_Password='+cn_com_Password+'&com_OrgID=AAGZ0TZZ4&com_RoamOrgID=&com_PassChangeDate=2011-09-09+09%3A30%3A48&com_OperType=OPRT10&com_Manager=0&com_Level=&com_ContactPhone=13922204911&com_MacAddress=08-00-27-44-EC-99&com_OnDuty=0&com_ShareStore=&com_BirthDay=&com_WorkDate=&com_CertID=&com_Sex=&com_EducationLevel=&com_TotalLevel=&com_SkillLevel=&com_TrainLevel=&com_ComityLevel=&com_Operator_type=0&com_NativeHome=&com_graduateDate=&com_IsMarray=0&com_PolityFace=&com_HomeAddress=&com_JobLive=&com_NowPostID=&com_BloodType=&com_Healthy=&com_Character=&com_Enjoyful=&com_PriSocietyRelation=&com_FamilyDes=&com_StarLevel=&com_AssessRec=&com_hr_status=1&com_Restrict_time=0&com_Start_time=0&com_End_time=24&com_Enable_gprs=0&com_Gprs_starttime=0&com_Gprs_endtime=0&com_Check_mac=0&com_Mac=&com_IPAddress=10.244.121.122&costartUsingTime=2011-03-18+00%3A00%3A00&coendUsingTime=2038-01-01+00%3A00%3A00&com_QueueID=&com_InvVersion=&com_areaID=&servNumber=15767570081&transmit=layoutAction.do%3Fmethod%3DshowLayout%26layoutId%3D1%26viewId%3D0&recType=&stopKeyValue=%D5%FD%B3%A3%CA%B9%D3%C3&remotemac=',cookies=cookies,headers=header3)
#     print s2.text
             
    cookies=cn1_cookies1(r.cookies,cn_phoneNum)
#     print cookies
    parmas={'width':'727','height':'14'}
    s2=r.post('http://10.252.17.241/nguniteview/layoutAction.do?method=showView&ownerType=1&viewId=208',data=parmas,cookies=cookies,headers=header3)
    soup2=BeautifulSoup(s2.text,'lxml')
    return soup2.select('.fv_tb_tr')[7].select('td')[0].text,soup2.select('.fv_tb_tr')[6].select('td')[0].text

In [ ]:
    
# 空格密码  58b69c6e23b02e145a31494f2accd449f23ba7792e1f86e445d9c0f388636f153f75cc1d3a68d3e12294c0d56238348be59f3be3e77977df33355b1c4e2766e7767e4c9e3c5cdcb36ae32b809a1033dc0fe53b7b22f8f3441bea52e5c6123dd10643703f4e1a33656248d148f968ea5e5deece5eadb1983484844b0d102cf187
# 870127 51d74591e40a21baa8101fe431416118aa15131a440fefc50ad8cef1d4986bca6f8dbfc629da64b1aeb820f8c9cbfeb0990f1c3ee845864863cf32d9299612a59e497929d3b0499b8053bd6d42295c854c2b247325355da22650c53a1882ea8f7a163e307c3580fa0043f4dcf49f6c8712c6eb9135e17124e7e4f549020a3bc6
cn_pwd=u'51d74591e40a21baa8101fe431416118aa15131a440fefc50ad8cef1d4986bca6f8dbfc629da64b1aeb820f8c9cbfeb0990f1c3ee845864863cf32d9299612a59e497929d3b0499b8053bd6d42295c854c2b247325355da22650c53a1882ea8f7a163e307c3580fa0043f4dcf49f6c8712c6eb9135e17124e7e4f549020a3bc6'
cn_phoneNum=u'13430354035;13430354064;13430354149;13430354165;13430354188;13430354224;13430354455;13430354459;13430354465;13430354478'

for i in cn_phoneNum.split(';'):
     print i,cn_liuliang(r,i,cn_pwd)
    
    


